
---

# 🖥️ **Local & Cloud Basics (GPU prereqs)**

---

## 🛠️ Tools (install once)

* **kubectl** → manage clusters
* **Docker** → build & run images
* **Helm** *(optional)* → package deployments
* ⚡ Enable faster builds → `export DOCKER_BUILDKIT=1`

---

## 💻 Local CPU (quickest dev)

```bash
kind create cluster
kubectl create ns llm
kubectl -n llm create deploy echo --image=ealen/echo-server --port=80
kubectl -n llm expose deploy echo --port=80
kubectl -n llm port-forward svc/echo 8080:80
```

👉 Test at `http://localhost:8080`

🍏 Mac (Apple Silicon): build for amd64

```bash
docker buildx build --platform linux/amd64 -t user/app:dev .
```

---

## ☁️ Cloud GPU (checklist)

1. Provision cluster + **GPU node pool** (EKS/GKE/AKS).
2. Install **NVIDIA device plugin**.
3. Mount model storage → **RWX PVC** (best) or RWO + initContainer.
4. Label/taint GPU nodes → schedule pods correctly.

**GPU pod snippet**

```yaml
resources:
  requests: { nvidia.com/gpu: 1, cpu: "2", memory: "16Gi" }
  limits:   { nvidia.com/gpu: 1, cpu: "4", memory: "20Gi" }
```

**Quick GPU sanity check**

```bash
kubectl run -it --rm gpu-test \
  --image=nvidia/cuda:12.3.2-base-ubuntu22.04 \
  --limits=nvidia.com/gpu=1 -- nvidia-smi
```

---

## 💾 Storage for Models

* **📂 RWX PVC** → shared, mount `/models:ro` (best).
* **📥 RWO + initContainer** → download per pod at startup.
* **⚡ HuggingFace cache** → keep on PVC to avoid re-downloads.

**PVC Example**

```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata: { name: models-pvc, namespace: llm }
spec:
  accessModes: [ReadWriteMany]
  resources: { requests: { storage: 200Gi } }
```

---

## 🔍 Basic Ops (verify cluster)

```bash
kubectl get nodes -o wide          # check nodes + GPU labels
kubectl get pods -A                # see all pods
kubectl -n llm logs deploy/echo    # check logs
kubectl -n llm describe deploy vllm
```

---
